In [39]:
from sklearn.linear_model import Ridge
import numpy as np
import time
import string
from copy import copy
import pickle

In [2]:
umlautDictionary = {u'Ä': 'Ae', u'Ö': 'Oe', u'Ü': 'Ue',  u'ä': 'ae', u'ö': 'oe', u'ü': 'ue', u'ß':u'ss'}
with open('Forrest Gump.srt') as f_srt:
    text = f_srt.read().decode('utf-8')

for uml, norm_letter in umlautDictionary.iteritems():
    text = text.replace(uml, norm_letter)

text = text.split('\r\n')
text = filter(lambda s: s != u'', text)

In [12]:
timestamps = filter(lambda s: '>' in s, text)

number_indecies = filter(lambda s: text[s].isdigit(), range(len(text)))
timestamp_indecies = filter(lambda s: text[s] in timestamps, range(len(text)))
text_indecies = filter(lambda s: not (s in number_indecies or s in timestamp_indecies) , range(len(text)))

assert len(timestamps) == len(number_indecies)

parsed_timestamps = map(lambda s: [time.strptime(s[:12], '%H:%M:%S,%f'), 
                            time.strptime(s[17:], '%H:%M:%S,%f')], timestamps)
print timestamps[0]
print parsed_timestamps[0][0].tm_hour, parsed_timestamps[0][0].tm_min, parsed_timestamps[0][0].tm_sec
print parsed_timestamps[0][1].tm_hour, parsed_timestamps[0][1].tm_min, parsed_timestamps[0][1].tm_sec

00:06:19,933 --> 00:06:24,404
0 6 19
0 6 24


In [29]:
table = string.maketrans("", "")
parsed_phrases = []

for phrase in phrases:
    phrase = phrase.encode('utf-8')
    phrase = phrase.translate(table,string.punctuation)
    phrase = phrase.translate(table, string.digits)
    phrase = phrase.lower().decode('utf-8')
    phrase = phrase.split(' ')
    phrase = filter(lambda s: s != u'', phrase)
    parsed_phrases.append(phrase)

In [30]:
parsed_phrases[:3]

[[u'hallo', u'mein', u'name', u'ist', u'forrestforrest', u'gump'],
 [u'moechten', u'sie', u'eine', u'praline'],
 [u'die', u'koennte', u'ich', u'pfundweise', u'essen']]

In [31]:
parsed_srt = zip(parsed_timestamps, parsed_phrases)
print parsed_srt[0]

([time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=6, tm_sec=19, tm_wday=0, tm_yday=1, tm_isdst=-1), time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=6, tm_sec=24, tm_wday=0, tm_yday=1, tm_isdst=-1)], [u'hallo', u'mein', u'name', u'ist', u'forrestforrest', u'gump'])


In [32]:
with open('parsed_srt.pickle', 'wb') as f:
    pickle.dump(parsed_srt, f)

In [33]:
with open('word2vec_german_sub.pickle') as f:
    word_to_vec_dict = pickle.load(f)

In [34]:
time_vectors = []
erros_word = []
parsed_words = 0

for phrase in parsed_phrases:
    time_vectors.append([])
    for word in phrase:
        try:
            time_vectors[-1].append(word_to_vec_dict[word])
            parsed_words += 1
        except KeyError:
            erros_word.append(word)

In [48]:
timestamps_vectors = zip(timestamps, time_vectors)

with open("timestamp_vectors.pickle", 'wb') as f:
    pickl